- Author: Benjamin Du
- Date: 2022-05-24 00:13:15
- Modified: 2022-05-28 22:37:28
- Title: Filter a Polars LazyFrame in Rust
- Slug: filter-a-polars-lazyframe-in-rust
- Category: Computer Science
- Tags: Computer Science, programming, Rust, LazyFrame, polars, filter, Expr


**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

## Tips and Traps

1.  `LazyFrame.filter` filters rows using an `Expr`
    while `DataFrame.filter` filters rows using a mask of the type `ChunkedArray<BooleanType>`.

In [2]:
:sccache 1
:dep polars = { version = "0.21.1", features = ["lazy", "parquet"] }

sccache: true


In [3]:
use polars::prelude::*;
use polars::df;

In [8]:
let frame = df![
    "names" => ["a", "b", "c"],
    "values" => [1, 2, 3],
    "values_nulls" => [Some(1), None, Some(3)]
].unwrap();
frame

shape: (3, 3)
┌───────┬────────┬──────────────┐
│ names ┆ values ┆ values_nulls │
│ ---   ┆ ---    ┆ ---          │
│ str   ┆ i32    ┆ i32          │
╞═══════╪════════╪══════════════╡
│ a     ┆ 1      ┆ 1            │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b     ┆ 2      ┆ null         │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c     ┆ 3      ┆ 3            │
└───────┴────────┴──────────────┘

Convert the DataFrame to a lazy one
and then filter rows using an `Expr`.

In [9]:
frame.lazy().filter(
    col("values").gt(lit::<i32>(1))
).collect()

Ok(shape: (2, 3)
┌───────┬────────┬──────────────┐
│ names ┆ values ┆ values_nulls │
│ ---   ┆ ---    ┆ ---          │
│ str   ┆ i32    ┆ i32          │
╞═══════╪════════╪══════════════╡
│ b     ┆ 2      ┆ null         │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c     ┆ 3      ┆ 3            │
└───────┴────────┴──────────────┘)

In [15]:
let frame = df![
    "names" => ["a", "b", "c"],
    "values" => [1, 2, 3],
    "values_nulls" => [Some(1), None, Some(3)]
].unwrap();
let lframe = frame.lazy();
[lframe.fetch(2), lframe.fetch(2)]

Error: use of moved value: `lframe`